# Expansión de Taylor (motivación para XGBoost)

La **expansión de Taylor** es una herramienta matemática que permite aproximar una función complicada usando un polinomio. En el caso de XGBoost, se utiliza para **aproximar la función de pérdida** con el fin de encontrar la mejor mejora en cada iteración.

## Intuición

Supón que tienes una función complicada, como una función de pérdida que depende del modelo y de los datos, y quieres entender su comportamiento local para mejorarla paso a paso. La expansión de Taylor permite aproximarla con un polinomio más manejable, utilizando derivadas.

En particular, XGBoost **utiliza una expansión de segundo orden** (es decir, hasta la derivada segunda), porque esto da más información que una expansión lineal (como la que usan métodos de gradiente tradicional) y sigue siendo computacionalmente eficiente.

## Fórmula general de Taylor

La expansión de Taylor de una función $f(x)$ alrededor de un punto $a$ es:

$$
f(x) \approx f(a) + f'(a)(x - a) + \frac{f''(a)}{2}(x - a)^2
$$

(omitimos términos de orden mayor porque no se usan en XGBoost)

Si tomamos $a = 0$ para simplificar (como suele hacerse), entonces:

$$
f(x) \approx f(0) + f'(0)x + \frac{f''(0)}{2}x^2
$$

Esto es justamente lo que usa XGBoost: aproxima la función de pérdida como un polinomio cuadrático alrededor del valor actual de la predicción.

## Ejemplo simple

Aproximemos $f(x) = x^3 - 3x^2 + 2x + 1$ alrededor de $x = 0$, usando solo hasta la segunda derivada:

### Paso 1: Derivadas

- $f(x) = x^3 - 3x^2 + 2x + 1$
- $f'(x) = 3x^2 - 6x + 2$
- $f''(x) = 6x - 6$

### Paso 2: Evaluar en $x = 0$

- $f(0) = 1$
- $f'(0) = 2$
- $f''(0) = -6$

### Paso 3: Aproximación

Usando Taylor de segundo orden:

$$
f(x) \approx 1 + 2x - 3x^2
$$

### Paso 4: Evaluar en $x = 0.1$

$$
f(0.1) \approx 1 + 0.2 - 0.03 = 1.17
$$

El valor real es $f(0.1) = 1.171$, muy cerca.

## Conclusión

Con solo la derivada primera y segunda, podemos aproximar muy bien el valor de una función cerca de un punto.

**Esto es justo lo que hace XGBoost:**  
- Usa la **derivada primera** de la pérdida para saber hacia dónde moverse (gradiente)  
- Usa la **derivada segunda** para saber qué tan agresivo o conservador debe ser ese movimiento (curvatura)

Este enfoque cuadrático permite hacer mejoras más precisas en cada iteración del boosting.


In [12]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

# Función original
def f(x):
    return x**3 - 3*x**2 + 2*x + 1

# Derivadas
def f_prime(x):
    return 3*x**2 - 6*x + 2

def f_double_prime(x):
    return 6*x - 6

# Aproximación de Taylor de orden 2 centrada en a
def taylor_orden2(x, a):
    f0 = f(a)
    f1 = f_prime(a)
    f2 = f_double_prime(a)
    return f0 + f1 * (x - a) + (f2 / 2) * (x - a)**2

# Rango de valores para x
x_vals = np.linspace(-1, 3, 300)

# Función para graficar
def plot_taylor(a):
    true_vals = f(x_vals)
    taylor_vals = taylor_orden2(x_vals, a)

    plt.figure(figsize=(8, 5))
    plt.plot(x_vals, true_vals, label='Función real: $x^3 - 3x^2 + 2x + 1$', linewidth=2)
    plt.plot(x_vals, taylor_vals, '--', label=f'Aprox. Taylor (orden 2, $a={a}$)', linewidth=2)
    plt.axvline(a, color='red', linestyle=':', label=f'$a={a}$', linewidth=1.5)
    plt.title(r"Aproximación de segundo orden:  $f(x) \approx f(0) + f'(0)x + \frac{f''(0)}{2}x^2$")
    plt.xlabel("x")
    plt.ylabel("f(x)")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Slider interactivo para a
interact(plot_taylor, a=FloatSlider(value=0, min=-1.0, max=2.5, step=0.1))


interactive(children=(FloatSlider(value=0.0, description='a', max=2.5, min=-1.0), Output()), _dom_classes=('wi…

<function __main__.plot_taylor(a)>

In [7]:
!pip install ipywidgets
